# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
documents = establishments.find_one()
pprint(documents)


In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of documents in result:", count)

# Display the first document in the results using pprint
First_document = establishments.find_one(query)
print("First Document")
pprint(First_document)

In [ ]:
results = establishments.find(query)
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows: {num_rows}")


# Display the first 10 rows of the DataFrame
df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
#checking the column type
sample_document = establishments.find_one()

# Check the types of each field in the sample document
if sample_document:
    print("Field Types in Sample Document:")
    for key, value in sample_document.items():
        print(f"{key}: {type(value).__name__}")
else:
    print("No documents found in the collection.")


In [ ]:
query_broad = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}}
documents_broad = list(establishments.find(query_broad))
print("Documents in London (broad query):")
pprint(documents_broad)


In [ ]:
total_count = establishments.count_documents({})
print("Total number of documents in the collection:", total_count)

london_count = establishments.count_documents({"LocalAuthorityName": {"$regex": "London", "$options": "i"}})
print("Number of establishments with LocalAuthorityName containing 'London':", london_count)

sample_documents = list(establishments.find().limit(5))  # Get a few sample documents
print("Sample Documents:")
pprint(sample_documents)




In [ ]:
query = {"RatingValue": {"$gte": 4 } }
rv = establishments.find_one(query)
pprint(rv)

In [ ]:
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "RatingValue": {"$gte": 4}  # Using $gte to match documents with RatingValue of 4 or higher
}

# Count matching documents
count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue 4 or higher:", count)

# Fetch and display first 10 matching documents
documents = list(establishments.find(query))
print("First 10 Documents:")
from pprint import pprint
pprint(documents[:10])  # Print the first 10 documents


In [ ]:
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "RatingValue": {"$in": [4,5]}
}

# Find all documents that match the query
documents = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue >= 4:", count)


In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive match for 'London'
    "RatingValue": {"$gte": 4}  # RatingValue greater than or equal to 4
}

# Find all documents that match the query
documents = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue >= 4:", count)

# Display the first 10 documents in the results using pprint
print("First 10 Documents:")
pprint(documents[:10])  # Print the first 10 documents


In [ ]:
# Convert the result to a Pandas DataFrame
documents = list(establishments.find(query))

# Create a DataFrame from the documents
london_df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print("Number of documents in DataFrame:", len(london_df))

# Display the first document
london_df.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

In [ ]:
#checking the column type
sample_document = establishments.find_one()

# Check the types of each field in the sample document
if sample_document:
    print("Field Types in Sample Document:")
    for key, value in sample_document.items():
        print(f"{key}: {type(value).__name__}")
else:
    print("No documents found in the collection.")


In [ ]:
# Print all documents to see their structure
#all_documents = list(establishments.find())
#print("All Documents:")
#for doc in all_documents:
#    print(doc)

# Check for documents with RatingValue of 5
#rating_value_5_docs = list(establishments.find({"RatingValue": 5}))
#print("Documents with RatingValue of 5:")
#for doc in rating_value_5_docs:
 #   print(doc)

# Check for documents within the latitude and longitude range
latitude_range_docs = list(establishments.find({
    "geocode.latitude": {
        "$gte": latitude - degree_search,
        "$lte": latitude + degree_search
    },
    "geocode.longitude": {
        "$gte": longitude - degree_search,
        "$lte": longitude + degree_search
    }
}))
print("Documents within latitude and longitude range:")
for doc in latitude_range_docs:
    print(doc)


In [ ]:
test_query = {
    "geocode.latitude": {"$exists": True},
    "geocode.longitude": {"$exists": True}
}

test_docs = list(establishments.find(test_query))
print("Documents with latitude and longitude fields:")
for doc in test_docs:
    print(doc)


In [ ]:
establishments.update_many(
    {"geocode.latitude": {"$type": "string"}},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)

establishments.update_many(
    {"geocode.longitude": {"$type": "string"}},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)


In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200  
longitude = 0.08384000   

query = {
    "RatingValue": 5,
    "geocode.latitude": {
        "$gte": latitude - degree_search,
        "$lte": latitude + degree_search
    },
    "geocode.longitude": {
        "$gte": longitude - degree_search,
        "$lte": longitude + degree_search
    }
}

# Step 3: Define sorting by hygiene score in ascending order
sort = [("scores.Hygiene", 1)]

# Print the results
results = list(establishments.find(query).sort(sort).limit(5))
print(results)



In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)
df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query =     {'$match': {'scores.Hygiene': 0}}

# 2. Groups the matches by Local Authority
group_query =     {'$group': {
        '_id': '$LocalAuthorityName',
        'count': {'$sum': 1}
    }
}

# 3. Sorts the matches from highest to lowest
sort_query =     {'$sort': {'count': -1}}

pipeline = [match_query,group_query,sort_query]
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
print(results[:10])


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
rows = len(hygiene_df)
print(rows)

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)